In [1]:
from bn_helpers.get_structures_print_tools import get_nets, printNet
from benchmarking.model_evaluator import dependency_test
from bn_helpers.constants import MODEL

nets = get_nets()
myNet = nets[0]
printNet(myNet)
print()

print(dependency_test(myNet, model=MODEL, max_tokens=1000))

Loading Netica
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

prompt:
 In this Bayesian Network:
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

Is changing the evidence of B going to change the probability of A? Why?
quiz:
 1. Is changing the evidence of B going to change the probability of A?
A. No
B. None of the above
C. Yes

2. Why are they d-connected?
A. They are d-connected through the path ['B', 'A']
B. They are d-connected through the path ['A', 'B']
C. None of the above
D. They are d-connected through the path ['C', 'D']
y_list:
 ['C', 'A']



TypeError: get_answer_from_ollama() got an unexpected keyword argument 'max_tokens'

In [1]:
from ollama_helper.ollama_helper import answer_this_prompt
print(answer_this_prompt("Hello, how are you?"))

{"answer":"I'm fine, thanks! How can I help you today?"}


In [13]:
from ollama import chat, ChatResponse
# from bn_helpers.bn_helpers import AnswerStructure
from pydantic import BaseModel
class AnswerStructure(BaseModel):
    answer: str

resp = chat(
    model=MODEL,
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
    format=AnswerStructure.model_json_schema(),
    stream=False,
    think=False,                 
    options={"temperature": 0, "num_predict": 256},
)

print("RAW:", resp.message.content)           # should be '{"answer": "..."}'
print("THINK:", getattr(resp.message, "thinking", None))  # likely None now

RAW: 
THINK: None


In [18]:
from pydantic import BaseModel

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider


class CityLocation(BaseModel):
    city: str
    country: str

ollama_model = OpenAIChatModel(
    model_name=MODEL,
    provider=OllamaProvider(base_url='http://localhost:11434/v1'),  
)
agent = Agent(ollama_model, output_type=CityLocation)

result = await agent.run('Where were the olympics held in 2012?')
print(result)
print(result.output)
print(result.output.city)
print(result.output.country)
#> city='London' country='United Kingdom'
print(result.usage())
#> RunUsage(input_tokens=57, output_tokens=8, requests=1)

AgentRunResult(output=CityLocation(city='London', country='United Kingdom'))
city='London' country='United Kingdom'
London
United Kingdom
RunUsage(input_tokens=138, output_tokens=82, requests=1)
